### Configuración

Primero se cargan las librerías necesarias para el desarrollo del _notebook_

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from yaml import safe_load

Ahora se definen rutas relativas al _notebook_, para una adecuada manipulación de archivos dentro del directorio raíz.

In [2]:
# Ruta a directorio raíz
ROOT = os.path.dirname(os.getcwd())

# Abrir archivo de configuración
with open(file=os.path.join(ROOT, 'config.yml'), mode='r') as _file:
    CONFIG = safe_load(_file)
_file.close()

# 1. Cadenas de Markov vía Monte Carlo

Los métodos de Cadenas de Markov vía Monte Carlo (**MCMC**) son utilizados para **aproximar** la distribución posterior de un aplicación bajo enfoques Bayesianos.

Por ejemplo, en el clásico ejemplo de inferir la probabilidad $\theta$ de que una moneda resulte _Heads_ dado un conjunto de lanzamientos de la moneda, se pueden usar tres enfoques para conseguirlo:

* **Solución analítica**: La distribución inicial se especifica con una función conjugada con su verosimilitud, lo cual resulta en una distribución posterior que tiene solución analítica. Desafortunadamente, existen de pocas a nulas aplicaciones que se puedan resolver de esta manera.
* **Poblando el espacio parametral**: Conocemos el rango de valores que puede tomar el parámetro $\theta$, con lo que se puede especificar un _gridspace_ que abarque el espacio parametral. La desventaja de este enfoque es que rápidamente se vuelve dificil poblar el espacio a medida que incremente el número de parámetros, este problema es conocido como la **maldición de dimensionalidad**
* Cuando la aproximación mediante un _gridspace_ no es factible, entonces se utiizan los métodos MCMC

Estos métodos suponen que la distribución inicial está especificada por una función que fácilmente puede ser evaluada, es decir, dado un valor $\theta$ fácilmente se puede determinar $p(\theta)$. Esto resulta en una aproximación de la distribución posterior en la forma de un número grande de valores de $\theta$ muestreados aleatoriamente de la distribución.

¿Cómo se pueden generar muestras representativas de la posterior sin conocerla?

## 1.1. Algoritmo Metrópolis

Revisar la **sección 7.2.** del libro _Doing Bayesian Data Analyis_, J. Kruschke, 2° Edición.

### 1.1.1. Caminata Aleatoria

Supón que una persona vive en un sistema de islas consecutivas, constántemente viaja de una a otra, y al final del día debe decidir si se mueve a una nueva isla o no, Esto se puede representar en tres decisiones:
* Se queda en la isla en la que está actualmente,
* Se mueve a la isla a la derecha.
* Se mueve a la isla a la izquierda.

El objetivo es visitar las islas proporcionalmente a la población de cada una de ellas, pero el problema es que no sabe la población de cada isla, y desconoce a ciencia cierta cuántas islas hay. Sin embargo, a medida que pasa por cada isla tiene capacidad de:
* Preguntar la población de la isla en la que se encuentra.
* Cuando se propone una nueva isla por visitar, se puede preguntar la población de la nueva isla.

La heurística para decidir si se mueve o no es la siguiente:
1. Lanza un volado con una moneda justa para escojer la nueva isla (derecha o izquierda).
2. Si la nueva isla tiene una población más grande, se mueve sin dudar.
3. Si la nueva isla tiene menor población, entonces decide si moverse con criterios probabilísticos dados por:
    $$p_{mov}=\frac{P_{nueva}}{P_{actual}}$$
    
    En donde:
    * $P_{actual}$: Población de la isla actual.
    * $P_{nueva}$: Población de la isla propuesta.
    * $p_{mov}$: Probabilidad de moverse.
4. Luego gira una perinola con valores marcados uniformemente a lo largo de su circunferencia.
    * Si el valor de la perinola es menor a $p_{mov}$, se mueve.
    * Si el valor de la perinola es mayor $p_{mov}$, se queda en la isla actual.

### 1.1.2 Ejemplo de caminata aleatoria

Consideraciones del problema:
* Número de islas: 7
* Las islas están indexadas consecutivamente, la isla más a la izquierda es $\theta=1$, y la más a la derecha es $\theta=7$.
* La población relativa de las islas crece linealmente de modo que $P(\theta)=\theta$-
* El viajero empieza en la isla número 4, pero esto lo desconoce.